In [ ]:
import matplotlib.pyplot as plt
import cv2
import glob
import os
import shutil
from cv2 import dnn_superres
import ipywidgets as widgets

import random
from IPython.display import clear_output

## see the image files

In [ ]:
dir_path = 'original_images'

files = os.listdir(dir_path)
files

In [ ]:
# count the original image number

num_files = len(files)
num_files

In [ ]:
# make list of new file names

file_name_list = []
for i in range(num_files):
    name_of_file = str(i).zfill(4)
    file_name_list.append(name_of_file)
    
print(*file_name_list)

## first image check

In [ ]:
# load image

img_test = cv2.imread(dir_path + '/' + files[0])
img      = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
# flip image

img_o = cv2.imread(dir_path + '/' + files[0])
img   = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
img_r = cv2.flip(img, 1)
plt.imshow(img_r)
plt.show()

In [ ]:
# get image size

height_o, width_o, channels = img.shape[:3]
print( width_o, height_o)

In [ ]:
# make directory for images made

os.makedirs('original_and_flipped_images', exist_ok=True)
os.makedirs('resized_images', exist_ok=True)

## add filpped images right side left

In [ ]:
for i in range(len(files)):

    img_o = cv2.imread(dir_path + '/' + files[i])
    img_r = cv2.flip(img_o, 1)
    
    cv2.imwrite('original_and_flipped_images/' + file_name_list[i] + '_1_original.png' , img_o)
    cv2.imwrite('original_and_flipped_images/' + file_name_list[i] + '_2_flipped_.png' , img_r)   

## determine how many zoom images to generate

In [ ]:
one_animation_duration = 1
one_animation_image_num = int(float(one_animation_duration) * 5)
print('number of images =  ', one_animation_image_num, '  pcs')

In [ ]:
zoom_rate = 7.0

In [ ]:
# crop test : reduce pixels from 4 sides

pixel_to_reduce_in_x = zoom_rate * one_animation_image_num
print('pixel_to_reduce_in_x = ', pixel_to_reduce_in_x)
pixel_to_reduce_h = int(pixel_to_reduce_in_x)
pixel_to_reduce_w = int(pixel_to_reduce_in_x * (2/3))

img_resize  = img[pixel_to_reduce_h : height_o - pixel_to_reduce_w, pixel_to_reduce_w : width_o - pixel_to_reduce_w]

height_rsz, width_rsz, channels = img_resize.shape[:3]
print('width, height =  ', width_rsz, height_rsz)
plt.imshow(img_resize)
plt.show()

In [ ]:
# zoom in function

def zoom_in(img, pixcel_reduce, animation_image_order):        # min pixcel_reduce = around 3 depends on the original size
    height_o, width_o, channels = img.shape[:3]
    
    pixel_to_reduce_in_x = pixcel_reduce * animation_image_order
    pixel_to_reduce_h = int(pixel_to_reduce_in_x)
    pixel_to_reduce_w = int(pixel_to_reduce_in_x * (2/3))
    
    img_resize  = img[pixel_to_reduce_h : height_o - pixel_to_reduce_w, pixel_to_reduce_w : width_o - pixel_to_reduce_w]
    img2 = cv2.resize(img_resize, (width_rsz, height_rsz))
    
    return img2 

In [ ]:
# make images for animation using all images

dir_path = 'original_and_flipped_images'
files = os.listdir(dir_path)

image_global_num = 0

for i in range(len(files)):

    original_image = files[i]
    img_o = cv2.imread(dir_path + '/' + original_image)[:,:,::-1]
    img   = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
    for operation_num in range(one_animation_image_num):
        image_continuous_num = str(image_global_num).zfill(3)
        if operation_num <= one_animation_image_num:
            img_resize = zoom_in(img, zoom_rate, operation_num)
        else:
            img_resize = zoom_in(img, zoom_rate, one_animation_image_num)
            
        img_ds = cv2.resize(img_resize, (512, 768))
        f_name = files[i].replace(".png", "")
        cv2.imwrite('resized_images/' + f_name + '_' + image_continuous_num + '.png' , img_ds)
        image_global_num += 1
    
    clear_output(True)
    print('processing in : ', i + 1, ' out of ', len(files))
            
print('Done!')